In [2]:
import openpyxl as xl
import requests 
from bs4 import BeautifulSoup

In [3]:
class Player:    
    def __init__(self, name, score, link, img):
        self.name = name
        self.score = score
        self.link = link
        self.img = img
        self.position = ""
        self.height = 0
        self.weight = 0
        self.dob = ""
        self.team = ""
        
    def setDetails(self, position, positionNo, height, weight, dob, team):
        self.position = position
        self.positionNo = positionNo
        self.height = height
        self.weight = weight
        self.dob = dob
        self.team = team
    
    def getName(self):
        return self.name

    def getScore(self):
        return self.score

    def getLink(self):
        return self.link

    def getImg(self):
        return self.img

    def getPosition(self):
        return self.position
    
    def getPositionNo(self):
        return self.positionNo

    def getHeight(self):
        return self.height

    def getWeight(self):
        return self.weight

    def getDob(self):
        return self.dob

    def getTeam(self):
        return self.team

In [4]:
class Date:    
    def __init__(self, day, month, year):
        self.day = day        
        self.month = month                
        self.year = year

    def getDay(self):
        return self.day

    def getMonth(self):
        return self.month

    def getYear(self):
        return self.year

In [5]:
# Function for scrape players from rugby pass

def scrapePlayers():
    
    playerList = []

    page = requests.get('https://index.rugbypass.com/rpi/six-nations/all/all/7-days/high-to-low/players/')
    content = BeautifulSoup(page.content, 'html.parser')
    players = content.find_all(class_='player-bar')
    
    for p in players:
        newPlayer = Player(p.find(class_='name').string, p.find(class_='display-score').string, p['href'], p.find('img')['src'])
        playerList.append(newPlayer)
    return playerList

In [6]:
# for x in scrapePlayers(): 
#     print(x.getName() + "\t--> " + x.getScore())

In [7]:
# Function for scrape player deatails from rugby pass

def scrapeDetails(player):
    page = requests.get(player.link)
    content = BeautifulSoup(page.content, 'html.parser')
    
    position = "-"
    
    positionNo = 0

    height = 0

    weight = 0

    day = 0

    month = 0

    year = 0

    team = "-"
    
    positions = content.find_all(class_='positions')
    
    for p in positions:
        position = p.get_text('div')[1:-1]
        positionList = ["Loose Head Prop", 
                        "Hooker", 
                        "Tight Head Prop", 
                        "Lock 4", 
                        "Lock 5", 
                        "Blindside Flanker", 
                        "Openside Flanker", 
                        "Number 8", 
                        "Scrum Half", 
                        "Fly Half", 
                        "Left Wing", 
                        "Inside Centre", 
                        "Outside Centre", 
                        "Right Wing", 
                        "Fullback"]
        positionNo = positionList.index(position) + 1

    measurements = content.find_all(class_='measurement')

    for m in measurements:
        if ('divHeightdiv' in m.get_text('div')):
            heightStr = m.get_text('div').replace("divHeightdiv", "")
            if (heightStr.replace(" ", "") != "---"):
                height = float(heightStr.replace(" ", "")[:-1])

        elif ('divWeightdiv' in m.get_text('div')):
            weightStr = m.get_text('div').replace("divWeightdiv", "")
            if (weightStr.replace(" ", "") != "---"):
                weight = int(weightStr.replace(" ", "")[:-2])


        elif ('divDOBdiv' in m.get_text('div')):
            date =  m.get_text('div').replace("divDOBdiv", "")
            date = date.replace(" ", "")
            if (date != "---"):
                day = int(date[0:2])
                monthStr = date[2:5]

                months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                for mon in months:
                    if mon in monthStr:
                        month = months.index(monthStr) + 1
                year = int(date[5:9])

    teams = content.find_all(class_='player-team')

    for t in teams:
        if ("England" in t.get_text()):
            team = "England"

        elif ("France" in t.get_text()):
            team = "France"

        elif ("Ireland" in t.get_text()):
            team = "Ireland"

        elif ("Italy" in t.get_text()):
            team = "Italy"

        elif ("Scotland" in t.get_text()):
            team = "Scotland"

        elif ("Wales" in t.get_text()):
            team = "Wales"

    dob = Date(day, month, year)
    player.setDetails(position, positionNo, height, weight, dob, team)

In [8]:
# code for create a new xl sheet for player details

wb_playerdetails = xl.Workbook()

sheet = wb_playerdetails.active
sheet.title = "Player Details"

sheet.cell(1, 1).value = "Name"
sheet.cell(1, 2).value = "RPI"
sheet.cell(1, 3).value = "Link"
sheet.cell(1, 4).value = "Img"
sheet.cell(1, 5).value = "Position"
sheet.cell(1, 6).value = "Position No"
sheet.cell(1, 7).value = "Height"
sheet.cell(1, 8).value = "Weight"
sheet.cell(1, 9).value = "DOB - Day"
sheet.cell(1, 10).value = "DOB - Month"
sheet.cell(1, 11).value = "DOB - Year"
sheet.cell(1, 12).value = "Team"

row = 2
for player in scrapePlayers():
    scrapeDetails(player)
    sheet.cell(row, 1).value = player.getName()
    sheet.cell(row, 2).value = player.getScore()
    sheet.cell(row, 3).value = player.getLink()
    sheet.cell(row, 4).value = player.getImg()
    sheet.cell(row, 5).value = player.getPosition()
    sheet.cell(row, 6).value = player.getPositionNo()
    sheet.cell(row, 7).value = player.getHeight()
    sheet.cell(row, 8).value = player.getWeight()
    sheet.cell(row, 9).value = player.getDob().getDay()
    sheet.cell(row, 10).value = player.getDob().getMonth()
    sheet.cell(row, 11).value = player.getDob().getYear()
    sheet.cell(row, 12).value = player.getTeam()
    
    print(sheet.cell(row, 1).value + " --> " + sheet.cell(row, 12).value)
    
    row = row + 1

wb_playerdetails.save('Players.xlsx')

print("End... ")

James Ryan --> Ireland
James Lowe --> Ireland
Jamie George --> England
Ben Toolis --> Scotland
Thomas Ramos --> France
George Kruis --> England
James Tracy --> Ireland
Iain Henderson --> Ireland
Mako Vunipola --> England
Jamison Gibson-Park --> Ireland
Owen Farrell --> England
Robbie Henshaw --> Ireland
Antoine Dupont --> France
Dave Kearney --> Ireland
Nick Tompkins --> Wales
Romain Ntamack --> France
Ross Byrne --> Ireland
Stuart McCloskey --> Ireland
Andrew Porter --> Ireland
Liam Williams --> Wales
Maro Itoje --> England
Stuart McInally --> Scotland
Tadgh Beirne --> Ireland
Grant Gilchrist --> Scotland
Cian Healy --> Ireland
Gael Fickou --> France
Caelan Doris --> Ireland
Selevasio Tolofua --> -
Conor Murray --> Ireland
Sean Maitland --> Scotland
Adam Hastings --> Scotland
Ollie Devoto --> England
Elliot Daly --> England
Henry Pyrgos --> Scotland
Rhys Carre --> Wales
Gareth Davies --> Wales
Jacob Stockdale --> Ireland
Hugo Keenan --> Ireland
CJ Stander --> Ireland
Teddy Thomas --> 